In [ ]:
!pip install orjson

In [27]:
!pip install PyMuPDF
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 7.3 MB/s eta 0:00:00:00:0100:01


In [7]:
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 439.6 kB/s eta 0:00:00 0:00:01m


In [23]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
from functools import partial
import random
from typing import Dict, Tuple, Callable
from PIL import Image, UnidentifiedImageError
from typing import List, Optional
import pypdf
import orjson
import jsonlines     
import fitz  # PyMuPDF


In [15]:
json_path = Path('/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl')

# Open the JSONL file for reading
with jsonlines.open(json_path) as reader:
    for line_number, line in enumerate(reader, start=1):
        # Process each line as needed
        print(line['pdf'])

print(line_number)


/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00001.pdf
/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00012.pdf
/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00034.pdf
/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/pdf/2201.00070.pdf
4


"""
Copyright (c) Meta Platforms, Inc. and affiliates.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.
"""
# Implements image augmentation

import albumentations as alb
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD


def alb_wrapper(transform):
    def f(im):
        return transform(image=np.asarray(im))["image"]

    return f


class Erosion(alb.ImageOnlyTransform):
    """
    Apply erosion operation to an image.

    Erosion is a morphological operation that shrinks the white regions in a binary image.

    Args:
        scale (int or tuple/list of int): The scale or range for the size of the erosion kernel.
            If an integer is provided, a square kernel of that size will be used.
            If a tuple or list is provided, it should contain two integers representing the minimum
            and maximum sizes for the erosion kernel.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, scale, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        if type(scale) is tuple or type(scale) is list:
            assert len(scale) == 2
            self.scale = scale
        else:
            self.scale = (scale, scale)

    def apply(self, img, **params):
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, tuple(np.random.randint(self.scale[0], self.scale[1], 2))
        )
        img = cv2.erode(img, kernel, iterations=1)
        return img


class Dilation(alb.ImageOnlyTransform):
    """
    Apply dilation operation to an image.

    Dilation is a morphological operation that expands the white regions in a binary image.

    Args:
        scale (int or tuple/list of int): The scale or range for the size of the dilation kernel.
            If an integer is provided, a square kernel of that size will be used.
            If a tuple or list is provided, it should contain two integers representing the minimum
            and maximum sizes for the dilation kernel.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, scale, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        if type(scale) is tuple or type(scale) is list:
            assert len(scale) == 2
            self.scale = scale
        else:
            self.scale = (scale, scale)

    def apply(self, img, **params):
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, tuple(np.random.randint(self.scale[0], self.scale[1], 2))
        )
        img = cv2.dilate(img, kernel, iterations=1)
        return img


class Bitmap(alb.ImageOnlyTransform):
    """
    Apply a bitmap-style transformation to an image.

    This transformation replaces all pixel values below a certain threshold with a specified value.

    Args:
        value (int, optional): The value to replace pixels below the threshold with. Default is 0.
        lower (int, optional): The threshold value below which pixels will be replaced. Default is 200.
        always_apply (bool, optional): Whether to always apply this transformation. Default is False.
        p (float, optional): The probability of applying this transformation. Default is 0.5.

    Returns:
        numpy.ndarray: The transformed image.
    """

    def __init__(self, value=0, lower=200, always_apply=False, p=0.5):
        super().__init__(always_apply=always_apply, p=p)
        self.lower = lower
        self.value = value

    def apply(self, img, **params):
        img = img.copy()
        img[img < self.lower] = self.value
        return img


train_transform = alb_wrapper(
    alb.Compose(
        [
            Bitmap(p=0.05),
            alb.OneOf([Erosion((2, 3)), Dilation((2, 3))], p=0.02),
            alb.Affine(shear={"x": (0, 3), "y": (-3, 0)}, cval=(255, 255, 255), p=0.03),
            alb.ShiftScaleRotate(
                shift_limit_x=(0, 0.04),
                shift_limit_y=(0, 0.03),
                scale_limit=(-0.15, 0.03),
                rotate_limit=2,
                border_mode=0,
                interpolation=2,
                value=(255, 255, 255),
                p=0.03,
            ),
            alb.GridDistortion(
                distort_limit=0.05,
                border_mode=0,
                interpolation=2,
                value=(255, 255, 255),
                p=0.04,
            ),
            alb.Compose(
                [
                    alb.Affine(
                        translate_px=(0, 5), always_apply=True, cval=(255, 255, 255)
                    ),
                    alb.ElasticTransform(
                        p=1,
                        alpha=50,
                        sigma=120 * 0.1,
                        alpha_affine=120 * 0.01,
                        border_mode=0,
                        value=(255, 255, 255),
                    ),
                ],
                p=0.04,
            ),
            alb.RandomBrightnessContrast(0.1, 0.1, True, p=0.03),
            alb.ImageCompression(95, p=0.07),
            alb.GaussNoise(20, p=0.08),
            alb.GaussianBlur((3, 3), p=0.03),
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)
test_transform = alb_wrapper(
    alb.Compose(
        [
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)


In [47]:
import albumentations as alb
from albumentations.pytorch import ToTensorV2

def alb_wrapper(transform):
    def f(im):
        return transform(image=np.asarray(im))["image"]

    return f

train_transform = alb_wrapper(
        alb.Compose(
        [
            alb.Compose(
                [alb.ShiftScaleRotate(shift_limit=0, scale_limit=(-.15, 0), rotate_limit=1, border_mode=0, interpolation=3,
                                    value=[255, 255, 255], p=1),
                alb.GridDistortion(distort_limit=0.1, border_mode=0, interpolation=3, value=[255, 255, 255], p=.5)], p=.15),
            # alb.InvertImg(p=.15),
            alb.RGBShift(r_shift_limit=15, g_shift_limit=15,
                        b_shift_limit=15, p=0.3),
            alb.GaussNoise(10, p=.2),
            alb.RandomBrightnessContrast(.05, (-.2, 0), True, p=0.2),
            alb.ImageCompression(95, p=.3),
            alb.ToGray(always_apply=True),
            alb.Normalize((0.7931, 0.7931, 0.7931), (0.1738, 0.1738, 0.1738)),
            # alb.Sharpen()
            ToTensorV2(),
        ]
    )
)
test_transform = alb.Compose(
    [
        alb.ToGray(always_apply=True),
        alb.Normalize((0.7931, 0.7931, 0.7931), (0.1738, 0.1738, 0.1738)),
        # alb.Sharpen()
        ToTensorV2(),
    ]
)

In [55]:

import numpy as np
from PIL import Image
import cv2
from torchvision import transforms


from PIL import ImageOps
from timm.models.swin_transformer import SwinTransformer
from torchvision.transforms.functional import resize, rotate

class enc():

    def __init__(self):
        self.input_size = [896,672]
        self.align_long_axis = True
        self.training = True

    def crop_margin(self, img: Image.Image) -> Image.Image:
        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box
        return img.crop((a, b, w + a, h + b))

    # def to_tensor(self, img: Image.Image):
    #     if self.training:
    #         return train_transform
    #     else:
    #         return test_transform
    def to_tensor(self, img: Image.Image):
        if self.training:
            # return train_transform(image = img)['image']
            return transforms.ToTensor()(img)
        else:
            return transforms.ToTensor()(img)
    def prepare_input(self, pdf_path:str, random_padding: bool = False):
        self.input_tensor = None
        if pdf_path is None:
            return
        
        # Convert PDF to images using PyMuPDF
        doc = fitz.open(pdf_path)
        for page_number in range(len(doc)):
            page = doc[page_number]
            image = page.get_pixmap()
            # Convert Pixmap to PIL Image
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)

            # crop margins
            try:
                img = self.crop_margin(img.convert("RGB"))
            except OSError:
                # might throw an error for broken files
                return
            if img.height == 0 or img.width == 0:
                return
            if self.align_long_axis and (
                (self.input_size[0] > self.input_size[1] and img.width > img.height)
                or (self.input_size[0] < self.input_size[1] and img.width < img.height)
            ):
                img = rotate(img, angle=-90, expand=True)
            img = resize(img, min(self.input_size))
            img.thumbnail((self.input_size[1], self.input_size[0]))
            delta_width = self.input_size[1] - img.width
            delta_height = self.input_size[0] - img.height
            if random_padding:
                pad_width = np.random.randint(low=0, high=delta_width + 1)
                pad_height = np.random.randint(low=0, high=delta_height + 1)
            else:
                pad_width = delta_width // 2
                pad_height = delta_height // 2
            padding = (
                pad_width,
                pad_height,
                delta_width - pad_width,
                delta_height - pad_height,
            )
            
            page_tensor = self.to_tensor(ImageOps.expand(img, padding))
            # page_tensor = self.to_tensor(img)
            # print(page_tensor)
            # page_tensor=page_tensor.to('cuda')
            # print(page_tensor.shape)
            if self.input_tensor is None:
               self.input_tensor = page_tensor
            else:
                self.input_tensor = torch.cat([self.input_tensor, page_tensor], dim=2)
        return self.input_tensor

In [56]:
import json
class pdfDataset(Dataset):

    def __init__(self, dataset_path, split: str = "train"):
        super().__init__()
        self.dataset_path = dataset_path
        self.split = split
        self.pdf_path = []
        self.latex_path = []    

        with jsonlines.open(self.dataset_path) as reader:
            for line_number, line in enumerate(reader, start=1):
                self.pdf_path.append(line['pdf'])
                self.latex_path.append(line['latex'])
        self.dataset_length = line_number

    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx):

        pdf_path = self.pdf_path[idx]
        latex_path = self.latex_path[idx]

        # print("pdf: ", pdf_path)
        # print("latex: ", latex_path)
        encoder = enc()
        input_tensor = encoder.prepare_input(pdf_path, random_padding=True)

        print("input_tensor: ", input_tensor)
        print("input_tensor.shape: ", input_tensor.shape)

        with open(latex_path, "rb") as f:
            gnd_truth_data = f.read()
            try:
                gnd_truth_data = gnd_truth_data.decode("utf-8")  # Try decoding with UTF-8
            except:
                gnd_truth_data = gnd_truth_data.decode("latin-1", errors="ignore")  # Fallback to Latin-1, ignore errors

        # print("gnd_truth_data: ", gnd_truth_data)
        tokenizer_out = self.nougat_model.decoder.tokenizer(
            gnd_truth_data,
            max_length=self.max_length,
            padding="max_length",
            return_token_type_ids=False,
            truncation=True,
            return_tensors="pt",
        )
        input_ids = tokenizer_out["input_ids"].squeeze(0)
        attention_mask = tokenizer_out["attention_mask"].squeeze(0)
        """      
        # randomly perturb ground truth tokens
        if self.split == "train" and self.perturb:
            # check if we perturb tokens
            unpadded_length = attention_mask.sum()
            while random.random() < 0.1:
                try:
                    pos = random.randint(1, unpadded_length - 2)
                    token = random.randint(
                        23, len(self.nougat_model.decoder.tokenizer) - 1
                    )
                    input_ids[pos] = token
                except ValueError:
                    break"""
        return input_tensor, input_ids, attention_mask



        # Save input_tensor to JSON file
        # json_path = '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'
        # with open(json_path, "w") as file:
        #     json.dump(input_tensor.tolist(), file)
        


        return pdf_path, latex_path

In [57]:
dataset = pdfDataset(dataset_path="/mnt/NAS/patidarritesh/Pdf_2_LaTeX/pdf_2_tex/dataset/root/train.jsonl")

In [58]:
for i, sample in enumerate(dataset):
    print("iteration::",i, sample)
    break



input_tensor:  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
input_tensor.shape:  torch.Size([3, 896, 5376])
data:  \documentclass{article}
\PassOptionsToPackage{numbers}{natbib}
\newtheorem{theorem}{Theorem}[section]
\newtheorem{lemma}[theorem]{Lemma}
     
\

In [ ]:
## Padding Details
import json
json_path = '/mnt/NAS/patidarritesh/Pdf_2_LaTeX/Experiments/dataloader.json'
with open(json_path, "r") as file:
    loaded_data = json.load(file)



for j in range(len(loaded_data[0])):
    count =0
    for i in range(len(loaded_data[0][0])):
        if loaded_data[0][j][i]!=0 and loaded_data[0][0][i]!=1:
            # print(loaded_data[0][0][i])
            count+=1
    print("count: ", count)

In [ ]:

"""class pdfDataset(Dataset):

    def __init__(self, 
                dataset_path,
                split: str = "traisn",
                root_name: str = "pdf",):
        super().__init__()
        self.dataset_path = dataset_path
        self.split = split
        self.root_name = root_name
        self.path_to_root = Path(dataset_path).parent
        self.root_name = root_name
        # self.path_to_root = self.path_to_root.parent

        # if not split in self.dataset_path:
        #     pti = self.path_to_root / (template % split + ".jsonl")
        #     if pti.exists():
        #         self.dataset_path = pti
        #     else:
        #         raise ValueError(f'Dataset file for split "{split}" not found: {pti}')
        
        self.dataset_file = None  # mulitprocessing
        
        if self.dataset_file is None:
            self.dataset_file = Path(self.dataset_path).open()
        line = self.dataset_file.readline()
        data: Dict = orjson.loads(line)

        pdf_path: Path = self.path_to_root / self.root_name / data.pop("pdf")
        latex_path: Path = self.path_to_root / self.root_name / data.pop("latex")

        self.dataset = {"pdf": pdf_path, "ground_truth": latex_path}
        self.dataset_length = len(self.dataset)


    def __len__(self):
        return self.dataset_length

    def __getitem__(self, idx: int):

        sample = self.dataset[idx]
        # print("idx:", idx)
        # print("sample:", sample)
        return sample
        # if sample is None:
        #     # if sample is broken choose another randomly
        #     return self[random.randint(0, self.dataset_length - 1)]
        # if sample is None or sample["image"] is None or prod(sample["image"].size) == 0:
        #     input_tensor = None
        # else:
        #     input_tensor = self.nougat_model.encoder.prepare_input(
        #         sample["image"], random_padding=self.split == "train"
                
        #     )

        # tokenizer_out = self.nougat_model.decoder.tokenizer(
        #     sample["ground_truth"],
        #     max_length=self.max_length,
        #     padding="max_length",
        #     return_token_type_ids=False,
        #     truncation=True,
        #     return_tensors="pt",
        # )
        # input_ids = tokenizer_out["input_ids"].squeeze(0)
        # attention_mask = tokenizer_out["attention_mask"].squeeze(0)
        # # randomly perturb ground truth tokens
        # if self.split == "train" and self.perturb:
        #     # check if we perturb tokens
        #     unpadded_length = attention_mask.sum()
        #     while random.random() < 0.1:
        #         try:
        #             pos = random.randint(1, unpadded_length - 2)
        #             token = random.randint(
        #                 23, len(self.nougat_model.decoder.tokenizer) - 1
        #             )
        #             input_ids[pos] = token
        #         except ValueError:
        #             break
        # return input_tensor, input_ids, attention_mask
    def __iter__(self):
        for i in range(self.dataset_length):
            yield self[i]"""

